# Piano Performance Evaluation - 3-Way Model Comparison (Colab)

Trains 3 models to prove multi-modal fusion advantage:
1. Audio-Only (MERT only)
2. MIDI-Only (MIDIBert only)
3. Fusion (MERT + MIDIBert)

**Dimensions**: 3 core (note_accuracy, rhythmic_precision, tone_quality)
**Sample size**: 10,000 training samples
**Expected time**: 1-1.5 hours total with optimizations (was 6-7 hours!)
**Goal**: Prove fusion beats both baselines by 15-20%

## Key Optimizations
- Audio backend: torchaudio (3-10x faster than librosa)
- Data location: Local SSD (10-30x faster than Google Drive)
- Parallel loading: num_workers=4 (was 0 for Drive)
- Result: **6x overall speedup!**

## Google Drive Structure

```
MyDrive/
  crescendai_data/
    all_segments/              # Audio segments
      *.wav
      midi_segments/
        *.mid
    annotations/
      synthetic_train_filtered.jsonl    # 91,865 samples
      synthetic_val_filtered.jsonl
      synthetic_test_filtered.jsonl

  crescendai_checkpoints/
    audio_10k/                 # Audio-only checkpoints
    midi_10k/                  # MIDI-only checkpoints  
    fusion_10k/                # Fusion checkpoints
```

## Setup

In [1]:
# HuggingFace Login
import os
os.environ.pop("HF_TOKEN", None)
os.environ.pop("HUGGINGFACEHUB_API_TOKEN", None)

from huggingface_hub import login, HfApi

try:
    import getpass as gp
    raw = gp.getpass("Paste your Hugging Face token (input hidden): ")
    token = raw.decode() if isinstance(raw, (bytes, bytearray)) else raw
    if not isinstance(token, str):
        raise TypeError(f"Unexpected token type: {type(token).__name__}")
    token = token.strip()
    if not token:
        raise ValueError("Empty token provided")
    login(token=token, add_to_git_credential=False)
    who = HfApi().whoami(token=token)
    print(f"✓ Logged in as: {who.get('name') or who.get('email') or 'OK'}")
except Exception as e:
    print(f"[HF Login] getpass flow failed: {e}")
    print("Falling back to interactive login widget...")
    login()
    try:
        who = HfApi().whoami()
        print(f"✓ Logged in as: {who.get('name') or who.get('email') or 'OK'}")
    except Exception as e2:
        print(f"[HF Login] Verification skipped: {e2}")

✓ Logged in as: Jai-D


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Verify data exists
import os
ANNOTATIONS_ROOT = '/content/drive/MyDrive/crescendai_data/annotations'

required_files = [
    f'{ANNOTATIONS_ROOT}/synthetic_train_filtered.jsonl',
    f'{ANNOTATIONS_ROOT}/synthetic_val_filtered.jsonl',
    f'{ANNOTATIONS_ROOT}/synthetic_test_filtered.jsonl',
]

print("Checking for data files...")
for f in required_files:
    if os.path.exists(f):
        print(f"✓ {os.path.basename(f)}")
    else:
        print(f"✗ MISSING: {f}")
        raise FileNotFoundError(f"Required file not found: {f}")

print("\n✓ All data files present")

In [ ]:
# Clone repo
!rm -rf /content/crescendai
!git clone https://github.com/Jai-Dhiman/crescendai.git /content/crescendai
%cd /content/crescendai/model
!git log -1 --oneline

In [ ]:
# Install uv (fast Python package manager)
!curl -LsSf https://astral.sh/uv/install.sh | sh

# Add to PATH for this session
import os
os.environ['PATH'] = f"{os.environ['HOME']}/.cargo/bin:{os.environ['PATH']}"

print("\n✓ uv installed")

In [ ]:
# Install dependencies
!uv pip install --system -e .

# Suppress warnings
import warnings
warnings.filterwarnings('ignore', message='divide by zero')
warnings.filterwarnings('ignore', category=SyntaxWarning)  # pydub regex warnings

import torch
import pytorch_lightning as pl
print(f"PyTorch: {torch.__version__}")
print(f"Lightning: {pl.__version__}")
print("✓ Dependencies installed")

## GPU Check

In [ ]:
!nvidia-smi

import torch
if not torch.cuda.is_available():
    print("\n⚠️  NO GPU! Enable GPU: Runtime → Change runtime type → T4 GPU")
    raise RuntimeError("GPU required")

print(f"\n✓ GPU: {torch.cuda.get_device_name(0)}")
print(f"✓ Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Download MERT model (cached after first download)
from transformers import AutoModel

print("Downloading MERT-95M (~380MB)...")
model = AutoModel.from_pretrained("m-a-p/MERT-v1-95M", trust_remote_code=True)
print("✓ MERT-95M cached")

del model
torch.cuda.empty_cache()

In [ ]:
# OPTIMIZATION STEP 1: Setup Colab Environment
# Fixes audio backends and configures optimal settings
print("Step 1: Setting up optimized Colab environment...")
!python scripts/setup_colab_environment.py

In [ ]:
## Data Optimization Workflow

**IMPORTANT**: This optimized workflow gives 10-30x speedup over reading from Google Drive!

**Steps**:
1. **Setup Environment** - Fix audio backends, install dependencies
2. **Copy Data to Local SSD** - Move files from Drive to fast local storage (~5-10 min one-time cost)
3. **Preflight Check** - Verify everything works before training

**Why this matters**:
- Google Drive I/O: 50-200ms latency per file
- Local SSD I/O: 0.1-1ms latency per file
- **Result**: Training goes from 6-7 hours → 1-1.5 hours!

**Cost**: ~10 minutes setup, saves 5-6 hours training time

In [ ]:
# OPTIMIZATION STEP 2: Copy Data to Local SSD
# This is the BIGGEST optimization (10-30x speedup!)
# Copies audio/MIDI from Google Drive to local /tmp/ for fast access
print("\nStep 2: Copying data from Google Drive to local SSD...")
print("This will take ~5-10 minutes but saves HOURS during training!\n")

!python scripts/copy_data_to_local.py \
    --drive-root /content/drive/MyDrive/crescendai_data \
    --local-root /tmp/crescendai_data \
    --subset \
    --subset-size 10000

print("\n✓ Data copied to local SSD!")
print("Training will now be 10-30x faster than reading from Drive")

In [ ]:
# OPTIMIZATION STEP 3: Comprehensive Preflight Check
# Tests everything before training to catch issues early
print("\nStep 3: Running comprehensive preflight check...\n")

!python scripts/preflight_check.py --config configs/experiment_10k.yaml

print("\nIf all checks passed, you're ready to train!")
print("Expected training time with optimizations:")
print("  - Audio-only:  ~20-30 min (was 2 hours)")
print("  - MIDI-only:   ~15-20 min (was 1.5 hours)")
print("  - Fusion:      ~30-40 min (was 2.5 hours)")
print("  - TOTAL:       ~1-1.5 hours (was 6-7 hours!)")

## Experiment 1: Audio-Only (~20-30 min with optimizations)

Training with audio features only (MERT-95M encoder)

In [ ]:
%%time
!python train.py --config configs/experiment_10k.yaml --mode audio

## Experiment 2: MIDI-Only (~15-20 min with optimizations)

Training with MIDI features only (MIDIBert encoder)

In [ ]:
%%time
!python train.py --config configs/experiment_10k.yaml --mode midi

## Experiment 3: Fusion (~30-40 min with optimizations)

Training with both audio and MIDI features (multi-modal fusion)

In [ ]:
%%time
!python train.py --config configs/experiment_10k.yaml --mode fusion

## Compare Results

Load all 3 trained models and compare performance on test set

In [ ]:
import pytorch_lightning as pl
from src.models.lightning_module import PerformanceEvaluationModel
from src.data.dataset import create_dataloaders
from pathlib import Path

# Load all 3 models
models = {}
for mode in ['audio', 'midi', 'fusion']:
    ckpt_dir = Path(f'/content/drive/MyDrive/crescendai_checkpoints/{mode}_10k')
    ckpts = list(ckpt_dir.glob('*.ckpt'))
    if ckpts:
        latest = sorted(ckpts)[-1]
        print(f"Loading {mode}: {latest.name}")
        models[mode] = PerformanceEvaluationModel.load_from_checkpoint(str(latest))
        models[mode].eval()
        models[mode] = models[mode].cuda()
    else:
        print(f"⚠️  No checkpoint found for {mode}")

# Create test dataloader
_, _, test_loader = create_dataloaders(
    train_annotation_path='/tmp/training_data/synthetic_train_filtered.jsonl',
    val_annotation_path='/tmp/training_data/synthetic_val_filtered.jsonl',
    test_annotation_path='/tmp/training_data/synthetic_test_filtered.jsonl',
    dimension_names=['note_accuracy', 'rhythmic_precision', 'tone_quality'],
    batch_size=8,
    num_workers=0,
    augmentation_config=None,
    audio_sample_rate=24000,
    max_audio_length=240000,
    max_midi_events=512,
)

# Evaluate each model
trainer = pl.Trainer(accelerator='auto', devices='auto', precision=16)
results = {}

for mode, model in models.items():
    print(f"\nEvaluating {mode}...")
    test_results = trainer.test(model, dataloaders=test_loader, verbose=False)
    results[mode] = test_results[0]

print("\n" + "="*70)
print("COMPARISON")
print("="*70)
print(f"{'Dimension':<25} {'Audio r':<12} {'MIDI r':<12} {'Fusion r':<12} {'Gain'}")
print("-"*70)

for dim in ['note_accuracy', 'rhythmic_precision', 'tone_quality']:
    audio_r = results.get('audio', {}).get(f'test_pearson_{dim}', 0)
    midi_r = results.get('midi', {}).get(f'test_pearson_{dim}', 0)
    fusion_r = results.get('fusion', {}).get(f'test_pearson_{dim}', 0)
    gain = fusion_r - max(audio_r, midi_r)
    
    print(f"{dim:<25} {audio_r:>11.3f} {midi_r:>11.3f} {fusion_r:>11.3f} {gain:>+11.3f}")

avg_gain = sum(
    results.get('fusion', {}).get(f'test_pearson_{dim}', 0) - 
    max(results.get('audio', {}).get(f'test_pearson_{dim}', 0),
        results.get('midi', {}).get(f'test_pearson_{dim}', 0))
    for dim in ['note_accuracy', 'rhythmic_precision', 'tone_quality']
) / 3

print("-"*70)
print(f"Average fusion gain: {avg_gain:+.3f} ({avg_gain*100:+.1f}%)")
print("="*70)

if avg_gain > 0.05:
    print("\n✓ SUCCESS: Fusion shows clear multi-modal advantage!")
else:
    print("\n⚠️  WARNING: Fusion gain is marginal. Check fusion implementation.")